In [1]:
from surprise import Dataset

In [2]:
data = Dataset.load_builtin('ml-100k') # load_from_df, load_from_file
type(data)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\shoseo/.surprise_data/ml-100k


surprise.dataset.DatasetAutoFolds

In [4]:
print( len(data.raw_ratings) )
data.raw_ratings[:10]

100000


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(data.raw_ratings, 
                  columns=['user', 'item', 'rate', 'id'])

In [7]:
print(df.shape)
df.head()

(100000, 4)


,user,item,rate,id
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [8]:
df.drop('id', axis=1, inplace=True)

In [9]:
df.head()

,user,item,rate
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


In [10]:
df2 =df.set_index(['user', 'item'])
df2.head()

,,rate
user,item,
196,242,3.0
186,302,3.0
22,377,1.0
244,51,2.0
166,346,1.0


In [11]:
# df3 = df2.unstack(fill_value=0)
df3 = df2.unstack()
df3.head()

rate                                               ...                    \
item    1   10  100 1000 1001 1002 1003 1004 1005 1006  ...  990 991 992  993   
user                                                    ...                     
1     5.0  3.0  5.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN NaN NaN  NaN   
10    4.0  NaN  5.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN NaN NaN  NaN   
100   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  3.0 NaN NaN  NaN   
101   3.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN NaN NaN  NaN   
102   3.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN NaN NaN  2.0   

                              
item 994 995 996 997 998 999  
user                          
1    NaN NaN NaN NaN NaN NaN  
10   NaN NaN NaN NaN NaN NaN  
100  NaN NaN NaN NaN NaN NaN  
101  NaN NaN NaN NaN NaN NaN  
102  NaN NaN NaN NaN NaN NaN  

[5 rows x 1682 columns]

In [12]:
df3.shape

(943, 1682)

In [13]:
df3.iloc[200:205, 200:205].fillna('')

rate                   
item 1179 118 1180 1181 1182
user                        
28                          
280         2         2    3
281                         
282                         
283

In [31]:
print(df3.shape)
print(df3.shape[0] * df3.shape[1])
print(df3.isnull().sum().sum()) # null 전체 합 # 평점이 매겨지지 않은 데이터
print(df3.isnull().sum().sum() / (df3.shape[0] * df3.shape[1])) # 비어있는 정도는 93%(거의 비어있음)

(943, 1682)
1586126
1486126
0.9369533063577546


In [14]:
from surprise.model_selection import KFold
from surprise import BaselineOnly, accuracy # BaselineOnly -> 추천 만드는 것 # accuracy -> RMSE, MSE .. 계산
import numpy as np

In [15]:
bsl_options = {
    'method': 'als',
    'n_epochs': 5,
    'reg_u': 12,
    'reg_i': 5
}

model = BaselineOnly(bsl_options)

np.random.seed(42)
accuracies = np.zeros(3) # 0을 3개의 배열로 만듬(오차 계산하기 위해)
cv = KFold(3)
for i, (trainset, testset) in enumerate(cv.split(data)):
    model.fit(trainset)
    predictions = model.test(testset)
    accuracies[i] = accuracy.rmse(predictions, verbose=True)
    
accuracies.mean()    

Estimating biases using als...
RMSE: 0.9419
Estimating biases using als...
RMSE: 0.9452
Estimating biases using als...
RMSE: 0.9446


0.9438942224301442

In [16]:
from surprise.model_selection import cross_validate

In [17]:
cross_validate(model, data, cv=5)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.93566615, 0.9417574 , 0.94765799, 0.94101401, 0.94324797]),
 'test_mae': array([0.74072242, 0.7456534 , 0.75064874, 0.74492599, 0.7473415 ]),
 'fit_time': (0.10893058776855469,
  0.1349189281463623,
  0.1469106674194336,
  0.1269240379333496,
  0.14890718460083008),
 'test_time': (0.13391876220703125,
  0.16789460182189941,
  0.1109311580657959,
  0.16789650917053223,
  0.1189267635345459)}

In [18]:
from surprise import KNNBasic # 거리 기반 유사도를 사용해서 비슷한 제품을 찾는 도구

In [19]:
sim_options = { # 유사도 지정 시 옵션
#     'name' : 'cosine' # msd, cosine, pearson 등을 지정할 수 있다. msd는 mean squared difference
    'name' : 'pearson'
}
# model = KNNBasic(k=10, sim_options=sim_options)
model = KNNBasic(k=10, min_k=3, sim_options=sim_options) # k = 이웃개수 , default=40 # 최소 k수
cross_validate(model, data, cv=3)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([1.0431447 , 1.04773395, 1.0448752 ]),
 'test_mae': array([0.83074422, 0.83225782, 0.83015733]),
 'fit_time': (2.229628562927246, 2.2156364917755127, 2.1936495304107666),
 'test_time': (3.396909236907959, 3.55481219291687, 3.518834114074707)}

In [38]:
from surprise import SVD # singular value decomposition, 특이값 분해 # 압축된 데이터 뽑아보자

In [39]:
svd = SVD(n_factors=50) # 압축해서 50개의 요소들만 봐서 유사도 확인 해보자
svd # matrix_factorization -> 행렬의 인수분해 (M*n = M*?+?*N)

In [36]:
cross_validate(svd, data) # test_rmse 0.93으로 줄음

{'test_rmse': array([0.93717335, 0.94395212, 0.92881943, 0.92842554, 0.94362822]),
 'test_mae': array([0.73822869, 0.741949  , 0.7333828 , 0.730933  , 0.74485413]),
 'fit_time': (3.1690495014190674,
  3.216020107269287,
  3.2360081672668457,
  3.1800425052642822,
  3.1590566635131836),
 'test_time': (0.19188189506530762,
  0.18688511848449707,
  0.2718327045440674,
  0.19088292121887207,
  0.18188834190368652)}

In [41]:
trainset = data.build_full_trainset() # 전체 데이터를 훈련 데이터로 사용하겠다! # 유사도 구할 때 평점 비어있는 데이터 (trainset)

In [42]:
svd.fit(trainset) # 평점 있는 데이터로 훈련

In [45]:
testset = trainset.build_anti_testset() # 예측을 위해 평점이 비어있는 데이터를 뽑아냄(testset) # 평점이 없는 데이터 추출, anti_dataset()

In [46]:
print(type(testset))
print(len(testset))

<class 'list'>
1486126


In [49]:
predictions = svd.test(testset) # 비어 있는 평점에 대한 예측

In [52]:
print(type(predictions))
print(len(predictions))
predictions[:10] # 196번 사용자가 302번 상품에 대해 4.27 평점을 남겼을 것이다. (r_ui는 사용자에 대한 전체 평점평균) # was_impossible -> true값이 있는지?

<class 'list'>
1486126


[Prediction(uid='196', iid='302', r_ui=3.52986, est=4.275597232344562, details={'was_impossible': False}),
 Prediction(uid='196', iid='377', r_ui=3.52986, est=2.663626110479011, details={'was_impossible': False}),
 Prediction(uid='196', iid='51', r_ui=3.52986, est=3.0480068644308065, details={'was_impossible': False}),
 Prediction(uid='196', iid='346', r_ui=3.52986, est=3.521760923070217, details={'was_impossible': False}),
 Prediction(uid='196', iid='474', r_ui=3.52986, est=4.337536378548281, details={'was_impossible': False}),
 Prediction(uid='196', iid='265', r_ui=3.52986, est=3.830480463132064, details={'was_impossible': False}),
 Prediction(uid='196', iid='465', r_ui=3.52986, est=3.7566364471219718, details={'was_impossible': False}),
 Prediction(uid='196', iid='451', r_ui=3.52986, est=3.237152168174854, details={'was_impossible': False}),
 Prediction(uid='196', iid='86', r_ui=3.52986, est=4.163258925925642, details={'was_impossible': False}),
 Prediction(uid='196', iid='1014', r_

In [ ]:
from collections import defaultdict # defaultdict는 키가 없으면 자동으로 생성

# dic['a'] = dic['a'] + 1 # 기존의 값이 없으면 문제! -> defaultdict로 해결

top_n = defaultdict(list) # 없는 것이 요청되면 list를 기본값으로 만들어라
for uid, iid, _, est, _ in predictions: # 3개만 변수로 쓴다.
    top_n[uid].append( (iid, est) ) # { 사용자 아이디: [(제품아이디, 평점)] } -> {key : [value1, value2]} # 비어있는 경우 비어있는 리스트로 자동 생성

top_n  #사용자id: 제품,평점

In [61]:
# top_n['196'] # 196번 사용자가 어떤 평가를 했는지~
for uid, ratings in top_n.items():
    ratings.sort(key = lambda x: x[1], reverse=True)
    top_n[uid] = ratings[:10] # 평점이 높은 상위 10개까지

In [64]:
for idx, (uid, ratings) in enumerate(top_n.items()):
    if idx == 10: break
    print(uid, [rating[0] for rating in ratings]) # 제품을 추천하기 위해 0번째 것을 가져온다, 만약 평점이 보고 싶다면 1 # 196번 사용자에게 추천할 제품은 ~~~~

196 ['408', '657', '134', '513', '64', '169', '318', '114', '479', '357']
186 ['408', '318', '483', '511', '114', '313', '22', '69', '174', '172']
22 ['64', '169', '408', '12', '56', '178', '199', '318', '657', '357']
244 ['134', '127', '483', '474', '654', '23', '285', '48', '170', '480']
166 ['173', '408', '169', '114', '483', '64', '272', '190', '172', '50']
298 ['408', '169', '515', '313', '480', '251', '64', '513', '12', '114']
115 ['169', '168', '179', '285', '408', '134', '483', '175', '474', '135']
253 ['174', '181', '178', '265', '408', '316', '251', '515', '313', '603']
305 ['603', '124', '657', '487', '498', '513', '136', '114', '488', '515']
6 ['603', '657', '654', '114', '48', '60', '198', '656', '144', '179']
